<a href="https://colab.research.google.com/github/maralhi/Sensors_models/blob/main/05_Model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Projects_ML/Project_Sensors/df_cl.csv'

df = pd.read_csv(file_path)

df.head()


Mounted at /content/drive


,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail,is_outlier_footfall,is_outlier_CS,is_outlier_Temperature,footfall_tempMode_interaction
0,0,7,7,1,6,6,36,3,1,1,False,False,True,0
1,190,1,3,3,5,1,20,4,1,0,False,False,True,190
2,31,7,2,2,6,1,24,6,1,0,False,False,True,217
3,83,4,3,4,5,1,28,6,1,0,False,False,True,332
4,640,7,5,6,4,0,68,6,1,0,True,False,True,4480


In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import pandas as pd

file_path = '/content/drive/MyDrive/Projects_ML/Project_Sensors/df_cl.csv'
df = pd.read_csv(file_path)

X = df[['VOC', 'AQ', 'footfall_tempMode_interaction']]
y = df['fail']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% training, 40% temp (test + val)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 50% validation, 50% test (of the temp set)

models = {
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),

}

final_model = models['XGBoost']

X_final_train = pd.concat([X_train, X_val])
y_final_train = pd.concat([y_train, y_val])

final_model.fit(X_final_train, y_final_train)

y_test_pred = final_model.predict(X_test)

print("Final Test Set Evaluation:")
print(classification_report(y_test, y_test_pred))


Final Test Set Evaluation:
              precision    recall  f1-score   support

           0       0.93      0.87      0.90       111
           1       0.84      0.91      0.87        78

    accuracy                           0.89       189
   macro avg       0.88      0.89      0.89       189
weighted avg       0.89      0.89      0.89       189



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:45:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


We choose XGBoost as the final model based on its superior validation performance.

In [ ]:
import joblib

xgb_model = models['XGBoost']

joblib.dump(xgb_model, 'xgboost_default_model.pkl')


['xgboost_default_model.pkl']

In [ ]:
import joblib


xgb_model = joblib.load('xgboost_default_model.pkl')

y_pred = xgb_model.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report

y_test_pred = xgb_model.predict(X_test)
print("Classification Report on Test Set:")
print(classification_report(y_test, y_test_pred))


Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.93      0.87      0.90       111
           1       0.84      0.91      0.87        78

    accuracy                           0.89       189
   macro avg       0.88      0.89      0.89       189
weighted avg       0.89      0.89      0.89       189

